In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/Users/an-chilu/Desktop/Temp 242A repo/Bank_Marketing_Split_dataset_with_allocations.csv')
# columns_keep = ['Age', 
# 'MaritalStatus',
# 'EducationLevel',
# 'AnnualIncome',
# 'NetWorth',
# 'AccountBalance',
# 'InvestmentPortfolioValue',
# 'SalaryCategory',
# 'RiskRating',
# 'CustomerSegment',
# 'HasMortgage',
# 'HasPersonalLoan']
df

,ClientID,Age,Gender,MaritalStatus,EducationLevel,EmploymentStatus,JobTitle,Region,SalaryCategory,CustomerSegment,...,DaysSinceLastContact,PreviousYearDeposit,MarketingScore,ResponsePropensity,TermDepositSubscribed,US_Equity,International_Equity,Bonds,REIT,Cash
0,1,46,Male,Divorced,Secondary,Employed,Services,East,Mid,Mass,...,475,0,1.7179,0.21,0,0.24,0.18,0.12,0.12,0.34
1,2,39,Female,Married,Secondary,Self-employed,Blue-collar,Central,Low,Premium,...,-1,0,1.6803,0.46,1,0.08,0.06,0.24,0.04,0.58
2,3,48,Male,Married,Secondary,Self-employed,Services,South,Mid,Mass,...,318,0,1.0510,0.22,1,0.16,0.12,0.18,0.08,0.46
3,4,59,Female,Married,Secondary,Employed,Student,South,Low,Mass,...,819,0,1.5676,0.30,0,0.16,0.12,0.18,0.08,0.46
4,5,38,Male,Married,Tertiary,Employed,Retired,North,Low,Mass,...,362,0,1.2918,0.33,0,0.16,0.12,0.18,0.08,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,56,Male,Single,Secondary,Employed,Technician,West,Mid,Standard,...,-1,0,1.5349,0.22,0,0.24,0.18,0.12,0.12,0.34
9996,9997,18,Male,Married,Secondary,Retired,Student,South,Low,Standard,...,279,0,1.4021,0.25,0,0.08,0.06,0.24,0.04,0.58
9997,9998,32,Male,Single,Tertiary,Self-employed,Blue-collar,West,Mid,Mass,...,232,0,1.2949,0.21,0,0.16,0.12,0.18,0.08,0.46
9998,9999,46,Female,Married,Tertiary,Unemployed,Student,North,Low,Mass,...,743,0,1.1767,0.30,1,0.08,0.06,0.24,0.04,0.58


In [3]:
object_list = df.select_dtypes(include=['object']).columns.tolist()
df_encoded = pd.get_dummies(df, columns = object_list, drop_first = True)
object_list_encode = df_encoded.select_dtypes(include = ['bool']).columns.tolist()
for i in object_list_encode:
    df_encoded[i] = df_encoded[i].astype(int)
df_encoded

,ClientID,Age,AnnualIncome,NetWorth,CreditScore,CreditLimit,AccountLengthYears,TenureWithBank,AccountBalance,NumBankProducts,...,LastContactMonth_May,LastContactMonth_Nov,LastContactMonth_Oct,LastContactMonth_Sep,LastContactDay_Mon,LastContactDay_Thu,LastContactDay_Tue,LastContactDay_Wed,PrevCampaignOutcome_Nonexistent,PrevCampaignOutcome_Success
0,1,46,50548.83,31921.38,600,27580.55,5,9,19323.46,4,...,0,0,0,1,0,1,0,0,0,0
1,2,39,28168.89,80356.77,712,24677.93,10,10,13554.93,4,...,0,0,1,0,0,1,0,0,1,0
2,3,48,35797.77,36070.52,458,18168.59,5,7,7029.21,2,...,0,0,0,0,0,1,0,0,0,0
3,4,59,10000.00,11767.30,645,14759.73,8,17,2181.64,2,...,0,0,0,0,0,0,1,0,0,1
4,5,38,24344.60,8545.45,556,17268.96,12,12,16225.93,2,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,56,33186.94,29091.33,720,23982.84,11,11,4540.09,3,...,0,1,0,0,0,0,0,0,1,0
9996,9997,18,16646.32,7332.09,648,20383.47,7,7,1753.65,2,...,1,0,0,0,0,0,1,0,0,0
9997,9998,32,31313.55,16451.52,693,22750.74,2,2,3386.26,1,...,0,0,0,1,0,0,1,0,0,0
9998,9999,46,16934.76,10085.57,583,18191.75,6,7,9393.04,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
y_value = []
X = df_encoded.drop(columns = ['US_Equity', 'International_Equity', 'Bonds', 'REIT', 'Cash'])
Y = df_encoded[['US_Equity', 'International_Equity', 'Bonds', 'REIT', 'Cash']]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [6]:
model = LinearRegression()
model.fit(x_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [9]:
y_pred = model.predict(x_test)

from sklearn.metrics import r2_score

r2_per_target = r2_score(y_test, y_pred, multioutput='raw_values')
print("R2 per target:", r2_per_target)

r2_overall = r2_score(y_test, y_pred, multioutput='uniform_average')
print("Overall R2:", r2_overall)


R2 per target: [0.96260653 0.96260653 0.96260653 0.96260653 0.96260653]
Overall R2: 0.962606533853528


In [11]:
osr = r2_score(y_test, y_pred)
print("OSR:", osr)
osr_all = r2_score(y_test, y_pred, multioutput='uniform_average')
print("OSR:", osr_all)


OSR: 0.962606533853528
OSR: 0.962606533853528
